##### The entire code must be run from top to bottom in order to avoid any problems.

### Deal with Loans Table

In [1]:
import pandas as pd

# The following variable must be set to False if we are to run the entire script on the "Test" data
isTrainData = True
fileName = "train" if isTrainData else "test"

# Convert Date from numerical to a Date format.
output = pd.read_csv('BankData/loan_' + fileName + '.csv', delimiter = ";")
output['date'] = 19000000 + output['date']
output['date'] = pd.to_datetime(output['date'], format = '%Y%m%d')
output['status'] = output['status'].replace(-1,'unsuccessful')
output['status'] = output['status'].replace(1,'successful')

# Change the Date variable
loans = output.rename(columns = {'date' : 'date_loan'}, inplace = False)
loans

,loan_id,account_id,date_loan,amount,duration,payments,status
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful
1,5316,1801,1993-07-11,165960,36,4610,successful
2,6863,9188,1993-07-28,127080,60,2118,successful
3,5325,1843,1993-08-03,105804,36,2939,successful
4,7240,11013,1993-09-06,274740,60,4579,successful
...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,successful
324,5625,3189,1996-12-15,222180,60,3703,unsuccessful
325,6805,8972,1996-12-21,45024,48,938,successful
326,7233,10963,1996-12-25,115812,36,3217,successful


### Deal with the Accounts Table

In [2]:
# Convert Date from numerical to a Date format.
output = pd.read_csv('BankData/account.csv', delimiter = ";")
output['date'] = 19000000 + output['date']
output['date'] = pd.to_datetime(output['date'], format = '%Y%m%d')

# Change the Date variable
output.rename(columns = {'date' : 'date_account', 'district_id' : 'district_id_account'}, inplace = True)
output.replace(to_replace = 'monthly issuance', value = 0, inplace = True)
output.replace(to_replace = 'issuance after transaction', value = 1, inplace = True)
output.replace(to_replace = 'weekly issuance', value = 2, inplace = True)
accounts = output
accounts

,account_id,district_id_account,frequency,date_account
0,576,55,0,1993-01-01
1,3818,74,0,1993-01-01
2,704,55,0,1993-01-01
3,2378,16,0,1993-01-01
4,2632,24,0,1993-01-02
...,...,...,...,...
4495,124,55,0,1997-12-28
4496,3958,59,0,1997-12-28
4497,777,30,0,1997-12-28
4498,1573,63,0,1997-12-29


### Merge the Accounts and Loans tables Together (Inner Join)

#### Note: This step also creates the difference between the loan date and the account date (in days)

In [3]:
loanAccounts = pd.merge(loans,accounts,on='account_id')
loanAccounts['days_between'] = (loanAccounts['date_loan'] - loanAccounts['date_account']).dt.days  # Get days difference
loanAccounts

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_account,frequency,date_account,days_between
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful,30,2,1993-03-22,105
1,5316,1801,1993-07-11,165960,36,4610,successful,46,0,1993-02-13,148
2,6863,9188,1993-07-28,127080,60,2118,successful,45,0,1993-02-08,170
3,5325,1843,1993-08-03,105804,36,2939,successful,12,0,1993-01-30,185
4,7240,11013,1993-09-06,274740,60,4579,successful,1,2,1993-02-14,204
...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,successful,72,0,1995-01-21,691
324,5625,3189,1996-12-15,222180,60,3703,unsuccessful,29,0,1995-11-29,382
325,6805,8972,1996-12-21,45024,48,938,successful,70,0,1996-05-21,214
326,7233,10963,1996-12-25,115812,36,3217,successful,16,0,1995-05-20,585


### Set up the Transactions Table (Compiled information)

##### This process usually takes some time (aprox 2-5 minutes)

In [4]:
output = pd.read_csv('BankData/trans_' + fileName + '.csv', delimiter=";")

# The columns that will be present in the final Transactions Table
column_names = ["account_id", "transaction_count", "max_amount", "min_amount", "finalBalance", "deltaBalance", "creditCount", "withdrawalCount", 'amount_std', 'balance_std', 'amount_mean', 'balance_mean', 'credit_std', 'credit_mean', 'withdrawal_std', 'withdrawal_mean']
transactions = pd.DataFrame(columns = column_names)

# ########################################################################################################


def acquireDelta(query):
    delta = 0
    for index, row in query.iterrows():
        if row['type'] == 'credit':
            delta = delta + lockRow.iloc[0]['amount']
        else:
            delta = delta - lockRow.iloc[0]['amount']
    return round(delta, 2)


# ########################################################################################################

# Acquire ALL distinct clients

account_id = output['account_id'].unique().tolist()

# For each client acquire the remaining variables
for accID in account_id:
    query = output.loc[output['account_id'] == accID]      # Returns the rows whose account_id == accID
    transactionCount = len(query.index)                  # Number of transactions per account
    max_amount = query['amount'].max()                   # Highest transaction amount ever done
    min_amount = query['amount'].min()                   # Smallest transaction amount ever done
    
    # Standard Deviation + Mean calculation
    
    amountSTD = 0
    balanceSTD = 0
    amountMean = 0
    balanceMean = 0
    if( len(query) > 1 ):
        amountSTD = round(query['amount'].std(), 2)
        balanceSTD = round(query['balance'].std(), 2)
        amountMean = round(query['amount'].mean(), 2)
        balanceMean = round(query['balance'].mean(), 2)
    if( len(query) == 1 ):
        balanceMean = round(query['balance'].iloc[0], 2)
        amountMean = round(query['amount'].iloc[0], 2)
    
    # Credit Deviation + Mean calculation
    queryR = query.loc[query['type'] == 'credit']
    creditSTD = 0
    creditMean = 0
    if( len(queryR) > 1 ):
        creditSTD = round(queryR['amount'].std(), 2)
        creditMean = round(queryR['amount'].mean(), 2)
    if( len(queryR) == 1 ):
        creditMean = round(queryR['amount'].iloc[0], 2)
    
    # Withdrawal Deviation + Mean calculation
    queryR = query.loc[query['type'] == 'withdrawal']
    withdrawalSTD = 0
    withdrawalMean = 0
    if( len(queryR) > 1 ):
        withdrawalSTD = round(queryR['amount'].std(), 2)
        withdrawalMean = round(queryR['amount'].mean(), 2)
    if( len(queryR) == 1 ):
        withdrawalMean = round(queryR['amount'].iloc[0], 2)
    
    # The following code acquires the finalBalance (smallest date)
    lockRow = query[query.date == query.date.max()]    # Get the row with the highest date value
    finalBalance = lockRow.iloc[0]['balance']
    
    # Get the "Delta" of each account
    deltaBalance = acquireDelta(query)
    
    
    # Get Credit and Withdrawal amonunts
    withdrawalCount = 0
    creditCount = 0
    
    if 'credit' in query.type.value_counts():
        creditCount = query.type.value_counts().credit.item()
    
    if 'withdrawal' in query.type.value_counts():
        withdrawalCount = query.type.value_counts().withdrawal.item()
    
    storeData = {'account_id':accID, 
                 'transaction_count':transactionCount, 
                 'max_amount':max_amount,
                 'min_amount':min_amount,
                 'finalBalance':finalBalance,
                 'deltaBalance':deltaBalance,
                 'creditCount':creditCount,
                 'withdrawalCount':withdrawalCount,
                 'amount_std' : amountSTD, 
                 'balance_std' : balanceSTD, 
                 'amount_mean' : amountMean, 
                 'balance_mean': balanceMean,
                 'credit_std' : creditSTD, 
                 'credit_mean' : creditMean, 
                 'withdrawal_std' : withdrawalSTD, 
                 'withdrawal_mean' : withdrawalMean}
    
    transactions = transactions.append(storeData, ignore_index=True)

transactions
#dataframe.to_csv("data/processed/test_transactions.csv", index=False, sep=';')

C:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,account_id,transaction_count,max_amount,min_amount,finalBalance,deltaBalance,creditCount,withdrawalCount,amount_std,balance_std,amount_mean,balance_mean,credit_std,credit_mean,withdrawal_std,withdrawal_mean
0,5270.0,53.0,67124.0,14.6,103745.7,-27900.0,22.0,28.0,19189.33,24870.23,17600.36,69954.57,24223.20,23558.29,14048.02,12885.27
1,11265.0,43.0,17920.0,14.6,21029.0,-1900.0,21.0,22.0,3862.23,6652.64,3714.90,22198.18,3880.56,4304.04,3848.85,3152.54
2,10364.0,45.0,21739.5,14.6,49083.3,-43479.0,21.0,24.0,6350.49,10171.03,5957.54,34272.21,7695.53,7551.74,4615.54,4562.63
3,3834.0,171.0,24000.0,14.6,28531.7,-414000.0,48.0,123.0,5017.81,8912.02,3618.33,23771.97,7256.61,6742.35,3075.70,2399.20
4,9307.0,143.0,51700.0,0.8,39204.7,-59500.0,63.0,76.0,11103.84,20303.82,6720.90,36080.39,14136.50,7938.84,7659.41,5228.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3365,2362.0,1.0,700.0,700.0,700.0,700.0,1.0,0.0,0.00,0.00,700.00,700.00,0.00,700.00,0.00,0.00
3366,528.0,1.0,300.0,300.0,300.0,300.0,1.0,0.0,0.00,0.00,300.00,300.00,0.00,300.00,0.00,0.00
3367,3190.0,1.0,1000.0,1000.0,1000.0,1000.0,1.0,0.0,0.00,0.00,1000.00,1000.00,0.00,1000.00,0.00,0.00
3368,1988.0,1.0,300.0,300.0,300.0,300.0,1.0,0.0,0.00,0.00,300.00,300.00,0.00,300.00,0.00,0.00


In [5]:
# Join the Transactions table with the Accounts + Loans table

compiledData = pd.merge(loanAccounts, transactions, on='account_id')
compiledData

# Clean memory ::
loanAccounts = None
transactions = None
account_id = None
accounts = None
loans = None

### Disposition Table handler + Card Table handler

#### This step acquires the number of owners associated to each Account

In [6]:
output = pd.read_csv('BankData/disp.csv', delimiter=";")
ownerListing = output.loc[output['type'] == 'OWNER']

freq = output['account_id'].value_counts().rename_axis('account_id').reset_index(name='num_users')
freq.sort_values('account_id')

disp = pd.merge(ownerListing, freq, on='account_id')
disp.drop(columns = ['type'], inplace = True)
disp

,disp_id,client_id,account_id,num_users
0,1,1,1,1
1,2,2,2,2
2,4,4,3,2
3,6,6,4,1
4,7,7,5,1
...,...,...,...,...
4495,13623,13931,11333,1
4496,13647,13955,11349,2
4497,13660,13968,11359,1
4498,13663,13971,11362,1


In [7]:
# Merge Disposition Table with the other merged contents:

compiledData = pd.merge(compiledData, disp, on='account_id')
compiledData

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_account,frequency,date_account,...,balance_std,amount_mean,balance_mean,credit_std,credit_mean,withdrawal_std,withdrawal_mean,disp_id,client_id,num_users
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful,30,2,1993-03-22,...,8330.87,5025.00,12250.00,3774.81,5025.00,0.00,0.00,2166,2166,1
1,5316,1801,1993-07-11,165960,36,4610,successful,46,0,1993-02-13,...,29122.06,11015.64,52083.86,13998.09,13523.16,14855.93,9111.27,2181,2181,1
2,6863,9188,1993-07-28,127080,60,2118,successful,45,0,1993-02-08,...,11520.18,5417.46,30060.95,6801.23,5009.73,5390.57,6097.00,11006,11314,1
3,5325,1843,1993-08-03,105804,36,2939,successful,12,0,1993-01-30,...,14151.26,8253.08,41297.48,9818.59,9254.60,4919.10,7168.10,2235,2235,1
4,7240,11013,1993-09-06,274740,60,4579,successful,1,2,1993-02-14,...,25256.67,18945.97,57188.21,24264.23,21255.93,13981.92,16801.00,13231,13539,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,successful,72,0,1995-01-21,...,11044.49,4895.97,44197.51,9143.30,6448.58,5002.16,3830.46,10813,11121,2
324,5625,3189,1996-12-15,222180,60,3703,unsuccessful,29,0,1995-11-29,...,26510.56,11725.49,55230.44,15547.01,13417.56,12458.77,10109.59,3855,3855,1
325,6805,8972,1996-12-21,45024,48,938,successful,70,0,1996-05-21,...,13151.51,7779.82,41994.91,12066.49,8544.93,7718.59,6974.43,10742,11050,1
326,7233,10963,1996-12-25,115812,36,3217,successful,16,0,1995-05-20,...,21971.16,13280.08,56646.52,17170.85,16554.99,12640.02,10667.40,13172,13480,1


In [8]:
# Push the "Card" data into the compiledData table ::

output = pd.read_csv('BankData/card_' + fileName + '.csv', delimiter=";")
output = output.rename(columns = {'type' : 'card_type'})
output = output.drop(columns = ['card_id', 'issued'])

# Merge it into the compiledData column ::

compiledVersion = pd.merge(compiledData, output, on='disp_id', how='left')
compiledVersion['card_type'] = compiledVersion['card_type'].fillna(value = "no card")
compiledVersion['card_type'] = compiledVersion['card_type'].map({'no card': 0, 'junior': 1, 'classic': 2, 'gold': 3})
compiledVersion

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_account,frequency,date_account,...,amount_mean,balance_mean,credit_std,credit_mean,withdrawal_std,withdrawal_mean,disp_id,client_id,num_users,card_type
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful,30,2,1993-03-22,...,5025.00,12250.00,3774.81,5025.00,0.00,0.00,2166,2166,1,0
1,5316,1801,1993-07-11,165960,36,4610,successful,46,0,1993-02-13,...,11015.64,52083.86,13998.09,13523.16,14855.93,9111.27,2181,2181,1,0
2,6863,9188,1993-07-28,127080,60,2118,successful,45,0,1993-02-08,...,5417.46,30060.95,6801.23,5009.73,5390.57,6097.00,11006,11314,1,0
3,5325,1843,1993-08-03,105804,36,2939,successful,12,0,1993-01-30,...,8253.08,41297.48,9818.59,9254.60,4919.10,7168.10,2235,2235,1,0
4,7240,11013,1993-09-06,274740,60,4579,successful,1,2,1993-02-14,...,18945.97,57188.21,24264.23,21255.93,13981.92,16801.00,13231,13539,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,successful,72,0,1995-01-21,...,4895.97,44197.51,9143.30,6448.58,5002.16,3830.46,10813,11121,2,0
324,5625,3189,1996-12-15,222180,60,3703,unsuccessful,29,0,1995-11-29,...,11725.49,55230.44,15547.01,13417.56,12458.77,10109.59,3855,3855,1,0
325,6805,8972,1996-12-21,45024,48,938,successful,70,0,1996-05-21,...,7779.82,41994.91,12066.49,8544.93,7718.59,6974.43,10742,11050,1,0
326,7233,10963,1996-12-25,115812,36,3217,successful,16,0,1995-05-20,...,13280.08,56646.52,17170.85,16554.99,12640.02,10667.40,13172,13480,1,0


### Client Table handler

In [9]:
import numpy as np

output = pd.read_csv('BankData/client.csv', delimiter=";")
output['owner_age'] = 2000 - ( 1900 + np.floor(output['birth_number']/10000) )
output['owner_sex'] = round(output['birth_number']/10000) - np.floor(output['birth_number']/10000)

output['owner_age'] = output['owner_age'].astype(int)
output['owner_sex'] = output['owner_sex'].astype(int)
output.rename(columns = {'district_id' : 'district_id_client'}, inplace = True)
output

compiledData = pd.merge(compiledVersion, output, on='client_id')
compiledData

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_account,frequency,date_account,...,withdrawal_std,withdrawal_mean,disp_id,client_id,num_users,card_type,birth_number,district_id_client,owner_age,owner_sex
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful,30,2,1993-03-22,...,0.00,0.00,2166,2166,1,0,475722,30,53,1
1,5316,1801,1993-07-11,165960,36,4610,successful,46,0,1993-02-13,...,14855.93,9111.27,2181,2181,1,0,680722,46,32,0
2,6863,9188,1993-07-28,127080,60,2118,successful,45,0,1993-02-08,...,5390.57,6097.00,11006,11314,1,0,360602,45,64,0
3,5325,1843,1993-08-03,105804,36,2939,successful,12,0,1993-01-30,...,4919.10,7168.10,2235,2235,1,0,405420,14,60,1
4,7240,11013,1993-09-06,274740,60,4579,successful,1,2,1993-02-14,...,13981.92,16801.00,13231,13539,1,0,780907,63,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,9030,1996-12-12,155616,48,3242,successful,72,0,1995-01-21,...,5002.16,3830.46,10813,11121,2,0,715410,72,29,1
324,5625,3189,1996-12-15,222180,60,3703,unsuccessful,29,0,1995-11-29,...,12458.77,10109.59,3855,3855,1,0,390320,29,61,0
325,6805,8972,1996-12-21,45024,48,938,successful,70,0,1996-05-21,...,7718.59,6974.43,10742,11050,1,0,575504,70,43,1
326,7233,10963,1996-12-25,115812,36,3217,successful,16,0,1995-05-20,...,12640.02,10667.40,13172,13480,1,0,530601,16,47,0


### DistrictTable handler

In [10]:
from sklearn import preprocessing

def normalize_Dataframe(dataframe):
    x = dataframe.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

output = pd.read_csv('BankData/district.csv', delimiter=";")
output = output.rename(columns = {'code' : 'district_id',
                     'no. of inhabitants' : 'inhabitants', 
                     'no. of municipalities with inhabitants < 499' : 'municipalities_499',
                     'no. of municipalities with inhabitants 500-1999' : 'municipalities_500-1999',
                     'no. of municipalities with inhabitants 2000-9999' : 'municipalities_2000-9999',
                     'no. of municipalities with inhabitants >10000' : 'municipalities_10000',
                     'no. of cities' : 'num_cities',
                     "unemploymant rate '95" : 'unemployment_95',
                     "unemploymant rate '96" : 'unemployment_96',
                     "no. of commited crimes '95" : 'crime_95',
                     "no. of commited crimes '96" : 'crime_96',
                    })

##### ====================== Obtain the approximate values for '?' ================================================

dist_df = pd.DataFrame(columns=["id", "difference"])
toAnalyse = output.copy()
toAnalyse = toAnalyse.drop(columns = ['district_id', 'district_name', 'region', 'crime_95', 'unemployment_95'])
toAnalyse = normalize_Dataframe(toAnalyse)

# Find the row index with the '?'
unknownRow = (output.loc[output['crime_95'] == '?'])['district_id'] - 1

idValue = 1
for row in toAnalyse.iterrows():
    distance = np.linalg.norm(toAnalyse.iloc[unknownRow] - toAnalyse.iloc[idValue - 1])
    dist_df = dist_df.append({'id':idValue, 'difference':distance}, ignore_index = True)
    idValue = idValue + 1

dist_df = dist_df.drop(unknownRow)
specificRow = dist_df[dist_df.difference == dist_df.difference.min()]
dist_df = dist_df.drop(specificRow.index)
specificRow2 = dist_df[dist_df.difference == dist_df.difference.min()]
dist_df = dist_df.drop(specificRow2.index)
##### ===========================================================================================================

output['crime_95'][unknownRow] = ( int(output['crime_95'][specificRow.id.iloc[0] - 1]) + int( output['crime_95'][specificRow2.id.iloc[0] - 1] ) ) / 2
output['unemployment_95'][unknownRow] = ( float(output['unemployment_95'][specificRow.id.iloc[0]]) + float(output['unemployment_95'][specificRow2.id.iloc[0]])) / 2.0

# Change dataset types
output['crime_95'] = output['crime_95'].astype(float)
output['unemployment_95'] = output['unemployment_95'].astype(float)

output['average_crime'] = round(( output['crime_95'] + output['crime_96'] )/2)
output['average_unemployment'] = round(( output['unemployment_95'] + output['unemployment_95'] )/2)

C:\Users\UP2018~2\AppData\Local\Temp/ipykernel_3444/975999107.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['crime_95'][unknownRow] = ( int(output['crime_95'][specificRow.id.iloc[0] - 1]) + int( output['crime_95'][specificRow2.id.iloc[0] - 1] ) ) / 2
C:\Users\UP2018~2\AppData\Local\Temp/ipykernel_3444/975999107.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['unemployment_95'][unknownRow] = ( float(output['unemployment_95'][specificRow.id.iloc[0]]) + float(output['unemployment_95'][specificRow2.id.iloc[0]])) / 2.0


In [11]:
# Merge Districts with the main dataset

compiledVersion = pd.merge(compiledData, output, left_on='district_id_client', right_on='district_id')
compiledVersion

,loan_id,account_id,date_loan,amount,duration,payments,status,district_id_account,frequency,date_account,...,num_cities,ratio_urban_inhabitants,avg_salary,unemployment_95,unemployment_96,enterpreneurs,crime_95,crime_96,average_crime,average_unemployment
0,5314,1787,1993-07-05,96396,12,8033,unsuccessful,30,2,1993-03-22,...,10,81.8,9650,3.38,3.67,100,2985.0,2804,2894.0,3.0
1,5316,1801,1993-07-11,165960,36,4610,successful,46,0,1993-02-13,...,10,73.5,8369,1.79,2.31,117,2854.0,2618,2736.0,2.0
2,6456,7123,1993-12-09,47016,12,3918,successful,64,1,1993-04-14,...,10,73.5,8369,1.79,2.31,117,2854.0,2618,2736.0,2.0
3,7249,11065,1994-06-12,69360,48,1445,successful,11,0,1993-03-19,...,10,73.5,8369,1.79,2.31,117,2854.0,2618,2736.0,2.0
4,6863,9188,1993-07-28,127080,60,2118,successful,45,0,1993-02-08,...,5,53.5,8390,2.28,2.89,132,2080.0,2122,2101.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,5148,940,1996-05-29,284280,60,4738,successful,37,0,1994-11-15,...,9,62.3,9065,4.46,5.39,123,4147.0,4166,4156.0,4.0
324,6667,8127,1996-07-15,155880,24,6495,successful,37,0,1994-10-13,...,9,62.3,9065,4.46,5.39,123,4147.0,4166,4156.0,4.0
325,5741,3754,1996-11-03,68544,24,2856,successful,37,2,1996-02-19,...,9,62.3,9065,4.46,5.39,123,4147.0,4166,4156.0,4.0
326,7036,10019,1996-09-25,194940,36,5415,successful,8,0,1995-06-15,...,8,69.4,11277,1.25,1.44,127,5179.0,4987,5083.0,1.0


# Export the CSV

In [12]:
compiledVersion.to_csv('compiledData_' + fileName + '.csv', index = False)